# Week 3 Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

# PART 1

>## *For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.*

>*Start by creating a new Notebook for this assignment.*

>*Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:*

>*To create the above dataframe:*

>*The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood*

>*Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.*

>*More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.*

>*If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.*

>*Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.*

>*In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.*

>*Submit a link to your Notebook on your Github repository. *

In [1]:
!pip install beautifulsoup4 #install beautifulsoup https://beautiful-soup-4.readthedocs.io/en/latest/

In [2]:
#!pip install beautifulsoup4 #install beautifulsoup https://beautiful-soup-4.readthedocs.io/en/latest/
import pandas as pd
import requests
import numpy as np 
from bs4 import BeautifulSoup

In [3]:
#Get url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text

## Scrape data from url using BeautifulSoup

In [4]:
Toronto_data = BeautifulSoup(source, 'html')

In [5]:
table=Toronto_data.find('table')

In [6]:
column_names=['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)

In [7]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [8]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Data Cleaning

In [9]:
#Remove not assigned Borough cells
df=df[df['Borough']!='Not assigned']
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
temp=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp=temp.reset_index(drop=False)
temp.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

merge = pd.merge(df, temp, on='Postalcode')
merge.drop(['Neighbourhood'],axis=1,inplace=True)
merge.drop_duplicates(inplace=True)
merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)
merge.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
merge.axes

[Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
             ...
              93,  94,  95,  96,  97,  98,  99, 100, 101, 102],
            dtype='int64', length=103),
 Index(['Postalcode', 'Borough', 'Neighbourhood'], dtype='object')]

In [12]:
merge.shape

(103, 3)

# PART 2

In [13]:
!pip install geocoder

In [14]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [15]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, merge, on='Postalcode')
geodata=geo_merged[['Postalcode','Borough','Neighbourhood','Latitude','Longitude']]

geodata.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# PART 3

In [17]:
#%matplotlib inline

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import os

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



## Determining Toronto Canada geograpical coordinates.

### Keep Boroughs with Toronto in name

In [20]:
#geodata.axes

In [21]:
#Filter on Boroughs that have Toronto in name
toronto_data=geodata[geodata['Borough'].str.contains("Toronto")]

toronto_data=toronto_data.reset_index(drop=True)

toronto_data.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [22]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Create map of Toronto using latitude and longitude values

In [23]:
#map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
#map_toronto 

## Map of Toronto with neighbourhoods superimposed on top.

In [24]:
map_toronto = folium.Map(location=[latitude, longitude], tiles='Stamen Toner', zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood, postalcode, in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood'], toronto_data['Postalcode']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='gold',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Define Foursquare Credentials and Version

In [25]:
CLIENT_ID = '355RLWAUP40MLINWD15PZXIETQH01XAJUNHRNPSDPI2WCACM' # your Foursquare ID
CLIENT_SECRET = 'PAZB1BAI5BMTESTMA3DSNVCQ04ZOWJC4DAHWZAOVS5QNTDSR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

## Explore the neighborhood in our dataframe.

In [26]:
toronto_data.loc[0, 'Neighbourhood']

'The Beaches'

In [27]:
neighbourhood_latitude = toronto_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = toronto_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


## Get the top 100 venues that are within a radius of 1000 meters.

In [28]:
LIMIT = 100
radius = 1000 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

In [29]:
results = requests.get(url).json()

#Remove the hash below if you want to view the results
#results

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues=nearby_venues.reset_index(drop=True)
nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
2,Beaches Bake Shop,Bakery,43.680363,-79.289692
3,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993
4,The Beech Tree,Gastropub,43.680493,-79.288846


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

76 venues were returned by Foursquare.


## Explore Neighborhoods

In [33]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=1000
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## List all Neighbourhoods

In [36]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

## Size of the resulting dataframe

In [37]:
print(toronto_venues.shape)
toronto_venues.tail()

(3186, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3181,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Murphy's Law,43.667319,-79.312656,Pub
3182,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Tim Hortons,43.662644,-79.309945,Coffee Shop
3183,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Breakfast Club,43.662811,-79.310174,Breakfast Spot
3184,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Carters Landing,43.662414,-79.309898,Bistro
3185,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Ashbridges Bay Yacht Club (ABYC),43.658210,-79.311004,Harbor / Marina


## Analyze each Neighbourhood

In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Berczy Park,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.010000,0.00000,0.00,0.000000,0.00,0.00000,0.00,0.00,0.000000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.01,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.020000,0.00000,0.00,0.000000,0.00,0.00000,0.00,0.00,0.000000,0.00
2,"Business reply mail Processing Centre, South C...",0.00,0.000000,0.000000,0.021277,0.00,0.00,0.00,0.00,0.000000,...,0.000000,0.00000,0.00,0.000000,0.00,0.00000,0.00,0.00,0.000000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.066667,0.066667,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.000000,0.00000,0.00,0.000000,0.00,0.00000,0.00,0.00,0.000000,0.00
4,Central Bay Street,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,0.030000,...,0.020000,0.00000,0.00,0.000000,0.00,0.00000,0.00,0.00,0.020000,0.00
5,Christie,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.020000,0.00000,0.00,0.010000,0.00,0.01000,0.00,0.00,0.000000,0.00
6,Church and Wellesley,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.000000,0.00000,0.01,0.000000,0.00,0.00000,0.01,0.00,0.020000,0.00
7,"Commerce Court, Victoria Hotel",0.00,0.000000,0.000000,0.020000,0.00,0.00,0.00,0.00,0.020000,...,0.020000,0.00000,0.00,0.000000,0.00,0.00000,0.00,0.00,0.000000,0.00
8,Davisville,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.010000,0.01000,0.00,0.010000,0.00,0.01000,0.00,0.00,0.010000,0.00
9,Davisville North,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.010000,0.01000,0.00,0.010000,0.00,0.01000,0.00,0.00,0.020000,0.00


## Sort venues descending

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Create the new dataframe and display the top 10 venues for each neighborhood.

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Park,Bakery,Restaurant,Grocery Store,Gym,Gastropub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Restaurant,Coffee Shop,Bar,Bakery,Furniture / Home Store,Gift Shop,Tibetan Restaurant,Performing Arts Venue,Supermarket
2,"Business reply mail Processing Centre, South C...",Park,Pizza Place,Brewery,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Sushi Restaurant,Bakery,Breakfast Spot,Beach
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Harbor / Marina,Dance Studio,Scenic Lookout,Sculpture Garden,Track,Park,Garden,Dog Run
4,Central Bay Street,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Park,Art Gallery,Café,Arts & Crafts Store,Mexican Restaurant,Plaza


## Cluster Neighbourhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [42]:
#toronto_grouped.axes

In [43]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 1, 0, 0, 0, 0, 4, 4], dtype=int32)

In [44]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Coffee Shop,Pizza Place,Pub,Beach,Japanese Restaurant,Breakfast Spot,Burger Joint,Sandwich Place,Bakery,Asian Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Café,Coffee Shop,Pub,Italian Restaurant,Fast Food Restaurant,Bank,Frozen Yogurt Shop,Bakery,Grocery Store
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Indian Restaurant,Coffee Shop,Café,Fast Food Restaurant,Beach,Park,Restaurant,Burrito Place,Sandwich Place,Grocery Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Bar,Café,Diner,Brewery,American Restaurant,Vietnamese Restaurant,Bakery,French Restaurant,Italian Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Coffee Shop,Café,Bookstore,Park,College Quad,Gym / Fitness Center,College Gym,Trail,Donut Shop,Dumpling Restaurant


## Visualizing the resulting clusters (using Stamen Toner tiles)

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], tiles='Stamen Toner',zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
## Analyzing individual clusters

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Café,Coffee Shop,Pub,Italian Restaurant,Fast Food Restaurant,Bank,Frozen Yogurt Shop,Bakery,Grocery Store
2,East Toronto,0,Indian Restaurant,Coffee Shop,Café,Fast Food Restaurant,Beach,Park,Restaurant,Burrito Place,Sandwich Place,Grocery Store
3,East Toronto,0,Coffee Shop,Bar,Café,Diner,Brewery,American Restaurant,Vietnamese Restaurant,Bakery,French Restaurant,Italian Restaurant
11,Downtown Toronto,0,Japanese Restaurant,Park,Gastropub,Diner,Café,Farm,Caribbean Restaurant,Performing Arts Venue,Botanical Garden,Steakhouse
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Burger Joint,Restaurant,Park,Sushi Restaurant,Men's Store,Bubble Tea Shop,Café,Caribbean Restaurant
14,Downtown Toronto,0,Coffee Shop,Gastropub,Japanese Restaurant,Theater,Italian Restaurant,Café,Hotel,Pizza Place,Gay Bar,Burrito Place
15,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Theater,Gastropub,Furniture / Home Store,Japanese Restaurant,Creperie,Cosmetics Shop
16,Downtown Toronto,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Park,Bakery,Restaurant,Grocery Store,Gym,Gastropub
17,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Park,Art Gallery,Café,Arts & Crafts Store,Mexican Restaurant,Plaza
18,Downtown Toronto,0,Café,Coffee Shop,Hotel,Theater,Art Gallery,Tea Room,Japanese Restaurant,Plaza,Sushi Restaurant,Gym


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Downtown Toronto,1,Coffee Shop,Café,Harbor / Marina,Dance Studio,Scenic Lookout,Sculpture Garden,Track,Park,Garden,Dog Run


In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Coffee Shop,Café,Bookstore,Park,College Quad,Gym / Fitness Center,College Gym,Trail,Donut Shop,Dumpling Restaurant


In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,3,Coffee Shop,Grocery Store,Park,Pie Shop,Metro Station,Breakfast Spot,Candy Store,Bistro,Sandwich Place,Office


In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,4,Coffee Shop,Pizza Place,Pub,Beach,Japanese Restaurant,Breakfast Spot,Burger Joint,Sandwich Place,Bakery,Asian Restaurant
5,Central Toronto,4,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Café,Restaurant,Dessert Shop,Sushi Restaurant,Gym,Pizza Place,Pharmacy
6,Central Toronto,4,Coffee Shop,Italian Restaurant,Sporting Goods Shop,Skating Rink,Café,Restaurant,Mexican Restaurant,Diner,Park,Pizza Place
7,Central Toronto,4,Italian Restaurant,Sushi Restaurant,Coffee Shop,Café,Fast Food Restaurant,Pizza Place,Indian Restaurant,Dessert Shop,Restaurant,Gym
8,Central Toronto,4,Coffee Shop,Grocery Store,Italian Restaurant,Gym,Café,Park,Thai Restaurant,Restaurant,Bank,Pub
9,Central Toronto,4,Coffee Shop,Park,Sushi Restaurant,Thai Restaurant,Italian Restaurant,Grocery Store,Gym / Fitness Center,Liquor Store,French Restaurant,Bagel Shop
13,Downtown Toronto,4,Coffee Shop,Theater,Café,Pub,Diner,Park,Restaurant,Bakery,Italian Restaurant,Breakfast Spot
22,Central Toronto,4,Italian Restaurant,Pharmacy,Café,Bank,Coffee Shop,Sushi Restaurant,Pilates Studio,Gastropub,Japanese Restaurant,Gym
23,Central Toronto,4,Park,Coffee Shop,Café,Bank,Italian Restaurant,Trail,Sushi Restaurant,Gym / Fitness Center,Pharmacy,Deli / Bodega
31,West Toronto,4,Café,Coffee Shop,Park,Bakery,Sushi Restaurant,Bar,Portuguese Restaurant,Restaurant,Pharmacy,Brewery
